In [2]:
from openai import OpenAI
from random import randint
from os import environ
from pathlib import Path
from json import loads
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: x, maxTries = 10):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

def listAnswerConversion(answer):
    result = loads(answer)
    assert isinstance(result, list)
    for item in result:
        assert isinstance(item, str)
    return result

In [3]:
from json import loads
def findSemanticTriples(subj):
    query = f'Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. Give me five examples of semantic triples that contain "{subj}" as subject and return them in an array formatted like [["sub1", "pred1", "obj1"], ["sub2", "pred2", "obj2"], ...]. Return nothing but the array without explanation.'
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert all(isinstance(triple, list) for triple in result)
        assert all(len(triple) == 3 for triple in result)
        assert all(isinstance(term, str) for triple in result for term in triple)
        return result
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    return answer

In [3]:
findSemanticTriples("Matter")

[['Matter', 'consists of', 'Atoms'],
 ['Matter', 'occupies', 'Space'],
 ['Matter', 'has', 'Mass'],
 ['Matter', 'can change', 'States'],
 ['Matter', 'interacts with', 'Energy']]

In [4]:

def findSemanticTriplesWithTopic(subj, topic):
    query = f'Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. Give me ten examples of semantic triples that contain "{subj}" as subject and return them in an array formatted like [["sub1", "pred1", "obj1"], ["sub2", "pred2", "obj2"], ...]. The topic of the triples should be "{topic}" Return nothing but the array without explanation.'
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert all(isinstance(triple, list) for triple in result)
        assert all(len(triple) == 3 for triple in result)
        assert all(isinstance(term, str) for triple in result for term in triple)
        return result
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    return answer

In [54]:
findSemanticTriplesWithTopic("Matter", "Supernova")

[['Matter', 'is ejected by', 'Supernova'],
 ['Matter', 'undergoes nuclear fusion during', 'Supernova'],
 ['Matter', 'is converted into energy in', 'Supernova'],
 ['Matter', 'forms neutron stars after', 'Supernova'],
 ['Matter', 'increases in temperature during', 'Supernova'],
 ['Matter', 'expands rapidly during', 'Supernova'],
 ['Matter', 'becomes ionized in', 'Supernova'],
 ['Matter', 'emits electromagnetic radiation in', 'Supernova'],
 ['Matter', 'forms dust particles after', 'Supernova'],
 ['Matter', 'contributes to the cosmic dust in', 'Supernova']]

In [5]:
def findQuestionWithSemanticTripleAnswer(topicArea):
    query = f'''
    Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. 
    Give me a question of the topc area "{topicArea}" that can be answered with a semantic triple.
    Return the question together with the semantic triple that answers it in the format ["question", ["sub", "pred", "obj"]].
    Return nothing but the array without explanation.'''
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert len(result) == 2
        assert isinstance(result[0], str)
        assert isinstance(result[1], list)
        assert len(result[1]) == 3
        assert all(isinstance(term, str) for term in result[1])
        return result
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    return answer

In [37]:
findQuestionWithSemanticTripleAnswer("Particle Physics")

['What do quarks combine to form?', ['Quarks', 'combine to form', 'Hadrons']]

In [7]:
def findNeighbouringTopics(topic):
    query = f'Give me five subtopics of the topic "{topic}" and three related topics. Return them all in an array formatted like ["sub1", "sub2", "sub3", "sub4", "sub5", "rel1", "rel2", "rel3"]. Return nothing but the array without explanation.'
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert len(result) == 8
        for item in result:
            assert isinstance(item, str)
        return result
    answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
    return answer

In [14]:
findNeighbouringTopics("Gravity")

['Theories of Gravity',
 'Gravitational Fields',
 'Effects of Gravity on Time',
 'Gravity in Astrophysics',
 'Experimental Tests of Gravity',
 'Relativity',
 'Orbital Mechanics',
 'Quantum Mechanics']

In [8]:
class TopicNavigator:
    def __init__(self, rootTopic):
        self.currentTopic = rootTopic
        self.topicPath = [rootTopic]
        self.rootTopic = rootTopic
        self.choices = None
    def generateChoices(self):
        if self.choices != None:
            return
        query = f'Give me five subtopics of the topic "{self.currentTopic}" and three related topics. Return them all in an array formatted like ["sub1", "sub2", "sub3", "sub4", "sub5", "rel1", "rel2", "rel3"]. Return nothing but the array without explanation.'
        def answerConversion(answer):
            result = loads(answer)
            assert isinstance(result, list)
            assert len(result) == 8
            for item in result:
                assert isinstance(item, str)
            return result
        answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        self.choices = answer + [self.topicPath[-1], self.rootTopic]
    def navigateToNextTopic(self, topicNumber):
        topicNumber = topicNumber - 1
        assert self.choices != None
        if topicNumber < 5:
            self.topicPath.append(self.currentTopic)
        elif topicNumber == 8:
            if len(self.topicPath) > 1:
                self.topicPath.pop()
        elif topicNumber == 9:
            self.topicPath = [self.rootTopic]
        self.currentTopic = self.choices[topicNumber]
        self.choices = None
    def getChoicesString(self):
        self.generateChoices()
        return "\n".join([f"{i+1}: {topic}" for i, topic in enumerate(self.choices)])
    def navigateOneStepInDirection(self, targetTopic):
        query = f'''Wich of the following topics is most related to the topic "{targetTopic}"?
        {self.getChoicesString()}
        return nothing but the number of the topic.'''
        def answerConversion(answer):
            ret = int(answer)
            assert 1 <= ret <= 10
            return ret
        answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        assert success
        self.navigateToNextTopic(answer)
        

In [58]:
t = TopicNavigator("Physics")

In [34]:
t.navigateToNextTopic(10)
print(t.getChoicesString())

1: Classical Mechanics
2: Electromagnetism
3: Quantum Mechanics
4: Thermodynamics
5: Relativity
6: Chemistry
7: Mathematics
8: Engineering
9: Physics
10: Physics


In [9]:
def navigateToTopic(targetTopic, topicNavigator, iterations):
    for i in range(iterations):
        topicNavigator.navigateOneStepInDirection(targetTopic)
        print(topicNavigator.currentTopic)

In [62]:
#t = TopicNavigator("Physics")
navigateToTopic("Top quark", t, 10)

Standard Model
Particle Physics
Standard Model
Particle Physics
Elementary Particles
Quarks
Types of Quarks
Quarks
Standard Model
Quantum Chromodynamics


In [30]:
class TripleNavigator:
    def __init__(self, startTriple, topic):
        self.currentTriple = startTriple
        self.choices = None
        self.topicNavigator = TopicNavigator(topic)
    def getCurrentTripleString(self):
        return "[" + ", ".join(self.currentTriple) + "]"
    def goOneStepTowartsQuestionAnswer(self, question):
        # Find the part of the triple that should be altered
        currentTriple = self.getCurrentTripleString()
        query = f'''Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object.
                The goal is to find a semantic triple that answers the question "{question}".
                This should be achieved by altering the current triple {currentTriple}.
                Two of the three parts of the triple should be altered, one should remain the same.
                Return A if the subject should remain the same, B if the predicate should remain the same, and C if the object should remain the same.
                Return nothing but the letter.'''
        def answerConversion(answer):
            assert answer in ["A", "B", "C"]
            return answer
        alteringOption, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        assert success
        # Adjust the topic of the triple
        tripleString = "[" + (self.currentTriple[0] if alteringOption == "A" else "???") + ", " + (self.currentTriple[1] if alteringOption == "B" else "???") + ", " + (self.currentTriple[2] if alteringOption == "C" else "???") + "]"
        query = f'''The semantic triple {tripleString} should be completed to answer the question "{question}".
                Therefore, a topic for the triple is needed. Select a topic from the following list:
                {self.topicNavigator.getChoicesString()}
                Return nothing but the number of the topic.'''
        def answerConversion(answer):
            ret = int(answer)
            assert 1 <= ret <= 10
            return ret
        topicIndex, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        assert success
        self.topicNavigator.navigateToNextTopic(topicIndex)
        # Complete the triple
        query = f'''Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object.
                Create ten different completed versions of the triple {tripleString}. The topic of the triples should be "{self.topicNavigator.currentTopic}".
                Return the completed triples in the format [["sub1", "pred1", "obj1"], ["sub2", "pred2", "obj2"], ...].
                Return nothing but the array without explanation.'''
        def answerConversion(answer):
            result = loads(answer)
            assert isinstance(result, list)
            assert all(isinstance(triple, list) for triple in result)
            assert all(len(triple) == 3 for triple in result)
            assert all(isinstance(term, str) for triple in result for term in triple)
            return result
        tripleSelection, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        assert success
        # Select the triple that is most likely to answer the question
        numberedTriplesString = "\n".join([f"{i+1}: [" + ", ".join(triple) + "]" for i, triple in enumerate(tripleSelection)])
        query = f'''Select the triple that is most likely to answer the question "{question}".
                {numberedTriplesString}
                Return nothing but the number of the triple.'''
        def answerConversion(answer):
            ret = int(answer)
            assert 1 <= ret <= len(tripleSelection)
            return ret
        tripleIndex, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        assert success
        self.currentTriple = tripleSelection[tripleIndex - 1]
        # Check if the triple answers the question
        query = f'''Does the triple {self.getCurrentTripleString()} answer the question "{question}"?
                Return Y or N.'''
        def answerConversion(answer):
            answer = answer.upper()
            assert answer in ["Y", "N"]
            return answer
        answer, success = tryRecieveAnswer(query, answerConversion = answerConversion)
        assert success
        return answer == "Y"


In [25]:
question, answer = findQuestionWithSemanticTripleAnswer("Particle accelerators")
print(question)
print(answer)

What do particle accelerators produce?
['Particle accelerators', 'produce', 'High-energy particles']


In [11]:
tn = TripleNavigator(["Star", "emits", "Light"], "Physics")

In [33]:
for i in range(10):
    finished = tn.goOneStepTowartsQuestionAnswer(question)
    print(tn.currentTriple)
    print(tn.topicNavigator.currentTopic)
    if finished:
        print("Found answer!")
        break

['Particle Accelerators', 'are used in', 'Cancer Treatment']
Applications of Particle Accelerators
['Particle Accelerators', 'generate', 'High-Energy Particles']
High-Energy Physics
Found answer!
